# Plotting of  Aluminosilicate Phase Diagram from Berman(1988) database of pure solid phases (standard state properties)
## Data are adjusted datapoints from Holdaway1993
## Concepts Demonstrated
* Loading a thermodynamic database
* Imposing experimental priors

## Import libraries ...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook
import matplotlib as mpl
import pandas as pd
pd.set_option('display.max_columns', None)
from scipy import stats
import scipy as sp
from scipy.optimize import least_squares

import os
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '/Users/msghiorso/ENKI/PhaseObjC')

import thermoengine
from thermoengine import phases
from thermoengine import model
from thermoengine import calibrate

import importlib


## Define reusable functions for analysis and visualization
* improve code readability
* reduce coding errors
* speed up debugging

In [ ]:
def get_nice_colors(Ncolor, whitefrac=0.0):
    cmap = plt.get_cmap(name='viridis')
    ind_a = np.int_(np.round(np.linspace(0,255,Ncolor)))
    col_a = np.array([cmap.colors[ind] for ind in ind_a])
    col_a = (1-whitefrac)*col_a+whitefrac*np.ones(col_a.shape)
    return col_a



In [ ]:
def plot_phase_diagram(rxns, rxn_eqns, fig=None, Tlims=[200, 1100], 
                       show_legend=True):
    if fig is None:
        fig = plt.figure()
        
    plt.figure(fig.number)
    colors = get_nice_colors(3)
    
    Tlims = np.array(Tlims)

    rxn_bnds = []
    T_bnds = []
    P_bnds = []

    for irxn, irxn_eqn, col in zip(rxns, rxn_eqns, colors):
        iTbnds, iPbnds = irxn.trace_boundary(Tlims=Tlims+273.15, Nsamp=100)
        T_bnds.append(iTbnds)
        P_bnds.append(iPbnds)
        istable = irxn.stability(iTbnds, iPbnds, rxns)
    
        # irxn_bnd, = plt.plot(iTbnds-273.15, iPbnds/1e3, lw=2, color=col)
        irxn_bnd, = plt.plot(iTbnds[istable]-273.15, iPbnds[istable]/1e3, '-',
                             lw=2, color=col)
        irxn_bnd_meta, = plt.plot(iTbnds[~istable]-273.15, iPbnds[~istable]/1e3, '--',
                             lw=2, color=col)
    
        rxn_bnds.append(irxn_bnd)
   
    

    plt.xlabel('Temp.  [C]')
    plt.ylabel('Press.  [kbar]')

    # plt.plot(data_adj['data']['T'][1:], data_adj['data']['P'][1:], 'k+')

    if show_legend:
        plt.legend(rxn_bnds, rxn_eqns)

    # plt.ylim(-0., 18)
    plt.xlim(Tlims)
    return fig

def plot_rxn_data(rxn_data, fig=None):
    if fig is None:
        fig = plt.figure()
        
    plt.figure(fig.number)

    for ind_rxn, col in enumerate(get_nice_colors(rxn_data.rxn_num)):   
        
        imask_fwd = (rxn_data.rxn['rxn_id']==ind_rxn) & (
            (rxn_data.rxn['rxn_dir']=='FWD') | (rxn_data.rxn['rxn_dir']=='FWD?'))
        imask_rev = (rxn_data.rxn['rxn_id']==ind_rxn) & (
            (rxn_data.rxn['rxn_dir']=='REV') | (rxn_data.rxn['rxn_dir']=='REV?'))
        
        
        plt.plot(rxn_data.conditions['T'][imask_fwd]-273.15, 
                 rxn_data.conditions['P'][imask_fwd]/1e3, 'v', 
                 color=col, ms=4, mfc='None')
        plt.plot(rxn_data.conditions['T'][imask_rev]-273.15, 
                 rxn_data.conditions['P'][imask_rev]/1e3, '^', 
                 color=col, ms=4, mfc='None')
    
    return fig


In [ ]:
data_adj = pd.read_excel(
    'phase-rxn-data/Al2SiO5-calib-data-reformat-Holdaway1993.xlsx',
    sheet_name=None)

In [ ]:
prior = pd.read_excel('phase-rxn-data/priors.xlsx')

In [ ]:
V0_prior = prior.loc[prior['param_name']=='V0'].reset_index(drop=True)
H0_prior = prior.loc[prior['param_name']=='H0'].reset_index(drop=True)
S0_prior = prior.loc[prior['param_name']=='S0'].reset_index(drop=True)

In [ ]:
for i in V0_prior.loc[[False,False,False]]:
    print(i)

In [ ]:
S0_prior

In [ ]:
H0_prior['param_value']

In [ ]:
H0_prior['param_error']

In [ ]:
rxn_data = calibrate.RxnData(data_adj)

display('reference: ', rxn_data.reference.head())
display('setup: ', rxn_data.setup.head())
display('conditions: ', rxn_data.conditions.head())
display('rxn: ', rxn_data.rxn.head())

In [ ]:
calibDB = calibrate.Database(rxn_data, ignore_kinetics=True, 
                             ref_energy_phases=['And'], phase_priors=prior)

In [ ]:
param_group_dH0 = calibDB.get_param_group(kind='phase',base='dH0')
calibDB.param_values(param_group=param_group_dH0)

In [ ]:
calibDB._get_ref_energy(calibDB.phases[2])

In [ ]:
H0_P0 = calibDB.phases[0].get_param_values(param_names='delta H')
H0_P1 = calibDB.phases[1].get_param_values(param_names='delta H')
H0_P2 = calibDB.phases[2].get_param_values(param_names='delta H')

[H0_P1-H0_P0, H0_P2-H0_P0]

In [ ]:
calibDB._param_values

In [ ]:
param_group = calibDB.get_param_group(kind='all', base='V0')
calibDB.add_free_params(param_group)

In [ ]:
param_values0 = calibDB.param_values()
param_values0

In [ ]:
calibDB.eval_model_costfun(param_values0)

In [ ]:
calibDB.get_param_group(kind='all', base='dH0')

In [ ]:
param_group = calibDB.get_param_group(kind='phase', base='dH0')
param_group

In [ ]:
calibDB.param_values(param_group=param_group, scale_params=False)

In [ ]:
param_group = calibDB.get_param_group(kind='all')
calibDB.param_scales(param_group=param_group)

In [ ]:
-2593.8--2589.9

In [ ]:
-2586.1--2589.9

In [ ]:
fig = plot_phase_diagram(calibDB.rxns, calibDB.rxn_eqns)
plot_rxn_data(calibDB.rxn_data, fig=fig);

In [ ]:
rxn_affinity = pd.Series(calibDB.rxn_affinity(), name='affinity')
rxn_affinity_err = pd.Series(calibDB.rxn_affinity_error(), name='affinity_err')
pd.concat((calibDB.rxn_data.rxn, rxn_affinity, rxn_affinity_err) , axis=1)

In [ ]:
(rxn_affinity/rxn_affinity_err).head()

In [ ]:
affinity = pd.Series(calibDB.rxn_affinity(), name='affinity')
affinity_err = .01*pd.Series(calibDB.rxn_affinity_error(), name='affinity_err')
affinity_thresh = 0*np.ones(affinity.shape)
rxn_dir = calibDB.rxn_data.rxn['rxn_dir']

np.exp(np.mean(calibrate.Stats.logprob_rxn_dir(
    rxn_dir, affinity, affinity_thresh, affinity_err)))

In [ ]:
param_group = calibDB.get_param_group(kind='all', base='V0')
print(param_group)
calibDB.add_free_params(param_group)

In [ ]:
param_group = calibDB.get_param_group(kind='phase')
print(param_group)
calibDB.add_free_params(param_group)

In [ ]:
param_values0 = calibDB.param_values(param_group=param_group)
print(param_values0)

In [ ]:
calibDB.get_param_group(free=True)

In [ ]:

calibDB.eval_model_costfun(param_values0)

In [ ]:
fit = calibDB.fit_model()

In [ ]:
fig = plot_phase_diagram(calibDB.rxns, calibDB.rxn_eqns)
plot_rxn_data(calibDB.rxn_data, fig=fig);

# Manual adjustment of parameters to check calib

In [ ]:
param_group = calibDB.get_param_group(kind='all', base='V0')
calibDB.add_free_params(param_group)

In [ ]:
param_vals = calibDB.param_values(param_group=param_group)
calibDB.set_param_values(1.5*param_vals, param_group=param_group)

In [ ]:
calibDB.param_values(param_group=param_group)

In [ ]:
ind = calibDB._get_param_group_index(param_group)

In [ ]:
calibDB._param_names[ind]

In [ ]:
calibDB._param_values

In [ ]:
calibDB.param_values(param_group=param_group)

In [ ]:
fig = plot_phase_diagram(calibDB.rxns, calibDB.rxn_eqns, Tlims=[200, 1100], 
                         show_legend=True)
fig = plot_rxn_data(calibDB.rxn_data, fig=fig)

In [ ]:
param_vals = calibDB.param_values(param_group=param_group)

In [ ]:
calibDB.set_param_values(param_vals)

In [ ]:
param_values0 = calibDB.param_values()

In [ ]:
rxn_affinity_err

In [ ]:
calibDB.param_names()

In [ ]:
calibDB.param_values()

In [ ]:
pd.concat((calibDB.rxn_data.rxn, rxn_affinity, rxn_affinity_err) , axis=1)


In [ ]:
np.exp(calibrate.Stats.logprob_rxn_dir(rxn_dir, affinity, affinity_thresh, 
                                affinity_err))